In [25]:
# necessary import
import xgboost as xgb # for gradient boosting classifier
import numpy as np # for matrices and numerical manipulations
import pandas as pd # for dataframes
import matplotlib.pyplot as plt # for plots
import seaborn as sns # for visualizing data
import matplotlib
import sklearn # for machine learning algorithms

from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from xgboost import XGBRegressor, XGBClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score, roc_auc_score, accuracy_score
import joblib  # Using joblib instead of pickle for compression
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [26]:
# # System versions
# print("Platform:", sys.platform)
# print("Python version:", sys.version)
# print("---" * 47)

# Libraries versions
print("matplotlib version:", matplotlib.__version__)
print("seaborn version:", sns.__version__)
print("xgboost version:", xgb.__version__)
print("sklearn version:", sklearn.__version__)
print("pandas version:", pd.__version__)
print("numpy version:", np.__version__)

matplotlib version: 3.10.7
seaborn version: 0.13.2
xgboost version: 3.1.1
sklearn version: 1.6.1
pandas version: 2.3.3
numpy version: 2.2.6


In [11]:
ecocrop = pd.read_csv('../data/cleaned_ecocrop.csv')
weather = pd.read_csv('../data/merged_aez_weather.csv')

In [12]:
print(ecocrop.shape, weather.shape)

(1187, 19) (59960, 6)


In [13]:
ecocrop.columns = ecocrop.columns.str.lower()
ecocrop.columns

Index(['unnamed: 0', 'comname', 'scientificname', 'dra', 'cat', 'fer', 'sal',
       'topmn', 'topmx', 'ropmn', 'ropmx', 'phopmn', 'phopmx', 'gmin', 'gmax',
       'tmin', 'tmax', 'rmin', 'rmax'],
      dtype='object')

In [14]:
ecocrop.drop(columns=['unnamed: 0'], inplace=True)
ecocrop.columns

Index(['comname', 'scientificname', 'dra', 'cat', 'fer', 'sal', 'topmn',
       'topmx', 'ropmn', 'ropmx', 'phopmn', 'phopmx', 'gmin', 'gmax', 'tmin',
       'tmax', 'rmin', 'rmax'],
      dtype='object')

In [15]:
weather.columns = weather.columns.str.lower().str.strip()
weather.columns

Index(['date', 'prectotcorr', 't2m', 'rh2m', 'allsky_sfc_sw_dwn', 'aez'], dtype='object')

In [ ]:
ecocrop.columns

In [31]:
print(weather['aez'].unique())
print(weather.columns)

['Highlands (Humid)' 'Upper Midlands (High Potential)'
 'Lower Midlands (Semi-Arid)' 'Coastal Lowlands (Humid)'
 'Arid Lowlands (Arid)']
Index(['date', 'prectotcorr', 't2m', 'rh2m', 'allsky_sfc_sw_dwn', 'aez'], dtype='object')


## AGGREGATE WEATHER DATA BY AEZ (Location)
 - Aggregating weather data per AEZ allows your model to understand these differences so it can: Predict rainfall more accurately, Recommend crops suited to local climate, Capture long-term climate patterns, Detect planting seasons, Understand local variability (e.g., wetter highlands vs drier lowlands)
- Calculate annual statistics per AEZ -> For each AEZ, you calculate long-term annual summaries: mean annual rainfall, mean annual temperature, max/min values, annual rainfall variability, annual temperature variability

- Get long-term averages for each AEZ -> These represent climatology — the typical long-term conditions of a location. E.g.:, Highland AEZs → cool & wet, Coastal AEZs → hot & humid, Eastern lowlands → hot & dry. Long-term AEZ averages help the model learn: “This AEZ tends to rain more regardless of month”, “This AEZ is always hotter than others”, “This zone has less variability in rainfall”
- Flatten column names
- Also get monthly patterns per AEZ (for planting time recommendations) -> Monthly patterns matter because they Help detect long rains and short rains seasons, Can recommend optimal planting months per AEZ, Improves model performance on seasonal predictions, Captures different rainfall cycles across Kenya’s zones.

### Extract year and month

In [32]:
weather['date'] = pd.to_datetime(weather['date'])
weather['year'] = weather['date'].dt.year
weather['month'] = weather['date'].dt.month


### Calculate annual statistics per AEZ

In [34]:
weather_annual = weather.groupby(['aez', 'year']).agg({
    'prectotcorr': 'sum',          # Total annual rainfall
    't2m': 'mean',                  # Average temperature
    'rh2m': 'mean',                 # Average humidity
    'allsky_sfc_sw_dwn': 'mean'     # Average solar radiation (sunshine hours proxy)
}).reset_index()

### Get long-term averages for each AEZ

In [35]:
weather_by_aez = weather_annual.groupby('aez').agg({
    'prectotcorr': ['mean', 'std', 'min', 'max'],
    't2m': ['mean', 'std', 'min', 'max'],
    'rh2m': ['mean', 'std'],
    'allsky_sfc_sw_dwn': ['mean', 'std']
}).reset_index()

### Flatten column names

In [36]:
weather_by_aez.columns = ['_'.join(col).strip('_') for col in weather_by_aez.columns]

print("\nWeather statistics by AEZ:")
print(weather_by_aez.to_string())



Weather statistics by AEZ:
                               aez  prectotcorr_mean  prectotcorr_std  prectotcorr_min  prectotcorr_max   t2m_mean   t2m_std    t2m_min    t2m_max  rh2m_mean  rh2m_std  allsky_sfc_sw_dwn_mean  allsky_sfc_sw_dwn_std
0             Arid Lowlands (Arid)        290.030909       122.887788           104.46           615.91  29.390140  0.447202  28.550685  30.187342  45.443923  2.520363               22.835453               0.596880
1         Coastal Lowlands (Humid)        882.669091       229.432192           463.00          1524.86  26.770129  0.288351  26.120685  27.341639  74.248891  1.838063               21.656370               0.401605
2                Highlands (Humid)       1272.039697       315.673400           675.18          2033.46  16.223052  0.418153  15.416339  16.995507  76.790897  3.678106               20.374437               0.457632
3       Lower Midlands (Semi-Arid)        675.810909       157.914689           407.60          1025.77  21.0144

### Also get monthly patterns per AEZ (for planting time recommendations)

In [38]:
weather_monthly = weather.groupby(['aez', 'month']).agg({
    'prectotcorr': 'mean',
    't2m': 'mean'
}).reset_index()

print("\nMonthly rainfall patterns by AEZ:")
print(weather_monthly.pivot(index='month', columns='aez', values='prectotcorr'))


Monthly rainfall patterns by AEZ:
aez    Arid Lowlands (Arid)  Coastal Lowlands (Humid)  Highlands (Humid)  \
month                                                                      
1                  0.438231                  0.901163           1.979316   
2                  0.314270                  0.581803           1.522747   
3                  0.775992                  1.901584           3.514956   
4                  1.747707                  4.165020           7.657455   
5                  1.329941                  6.011828           4.729022   
6                  0.571768                  2.374253           2.127000   
7                  0.551417                  1.246608           2.053118   
8                  0.787370                  0.880538           2.606109   
9                  0.357475                  1.279475           2.103909   
10                 0.719003                  2.792493           4.166931   
11                 1.204052                  4.297469

## CREATE TRAINING DATASET

In [42]:
# for each crop-aez combination, determine suitability
training_data = []

for _, crop in ecocrop.iterrows():
    for _, aez_row in weather_by_aez.iterrows():
        aez_name = aez_row['aez']
        avg_temp = aez_row['t2m_mean']
        avg_rainfall = aez_row['prectotcorr_mean']
        avg_humidity = aez_row['rh2m_mean']
        avg_solar = aez_row['allsky_sfc_sw_dwn_mean']
        
        # check suitability based on optimal ranges
        temp_optimal = (avg_temp >= crop['topmn']) & (avg_temp <= crop['topmx'])
        temp_absolute = (avg_temp >= crop['tmin']) & (avg_temp <= crop['tmax'])
        
        rain_optimal = (avg_rainfall >= crop['ropmn']) & (avg_rainfall <= crop['ropmx'])
        rain_absolute = (avg_rainfall >= crop['rmin']) & (avg_rainfall <= crop['rmax'])
        
        # suitability scoring
        if temp_optimal and rain_optimal:
            suitability = 2  # highly suitable
        elif temp_absolute and rain_absolute:
            suitability = 1  # marginally suitable
        else:
            suitability = 0  # not suitable
        
        training_data.append({
            'crop_name': crop['comname'],
            'scientific_name': crop['scientificname'],
            'aez': aez_name,
            # climate features from location
            'avg_temperature': avg_temp,
            'avg_rainfall': avg_rainfall,
            'avg_humidity': avg_humidity,
            'avg_solar_radiation': avg_solar,
            'temp_variability': aez_row['t2m_std'],
            'rainfall_variability': aez_row['prectotcorr_std'],
            # crop requirements
            'crop_temp_opt_min': crop['topmn'],
            'crop_temp_opt_max': crop['topmx'],
            'crop_temp_abs_min': crop['tmin'],
            'crop_temp_abs_max': crop['tmax'],
            'crop_rain_opt_min': crop['ropmn'],
            'crop_rain_opt_max': crop['ropmx'],
            'crop_rain_abs_min': crop['rmin'],
            'crop_rain_abs_max': crop['rmax'],
            'crop_ph_min': crop.get('phopmn', 5.5),
            'crop_ph_max': crop.get('phopmx', 7.5),
            'growth_days_min': crop.get('gmin', 60),
            'growth_days_max': crop.get('gmax', 120),
            # target
            'suitability': suitability
        })

df_training = pd.DataFrame(training_data)

print(f"training dataset shape: {df_training.shape}")
print(f"\nsuitability distribution:")
print(df_training['suitability'].value_counts())
print(f"\nsuitability by aez:")
print(df_training.groupby('aez')['suitability'].value_counts().unstack(fill_value=0))


training dataset shape: (5935, 22)

suitability distribution:
suitability
0    2761
1    2475
2     699
Name: count, dtype: int64

suitability by aez:
suitability                         0    1    2
aez                                            
Arid Lowlands (Arid)             1051  115   21
Coastal Lowlands (Humid)          365  584  238
Highlands (Humid)                 346  708  133
Lower Midlands (Semi-Arid)        453  489  245
Upper Midlands (High Potential)   546  579   62


In [40]:
weather.columns

Index(['date', 'prectotcorr', 't2m', 'rh2m', 'allsky_sfc_sw_dwn', 'aez',
       'year', 'month'],
      dtype='object')

## ENCODE CATEGORICAL VARIABLES

In [65]:
le_crop = LabelEncoder()
df_training['crop_encoded'] = le_crop.fit_transform(df_training['crop_name'])
print(f"Number of unique crops: {len(le_crop.classes_)}")

Number of unique crops: 1178


### Encode AEZ Location

In [67]:
# Encode AEZ (Location) - THIS IS A KEY FEATURE
le_aez = LabelEncoder()
df_training['aez_encoded'] = le_aez.fit_transform(df_training['aez'])


In [68]:

# One-hot encode AEZ for better model performance
ohe_aez = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
aez_onehot = ohe_aez.fit_transform(df_training[['aez']])
aez_onehot_df = pd.DataFrame(
    aez_onehot,
    columns=[f'_aez_{cat}' for cat in ohe_aez.categories_[0]],
    index=df_training.index
)
df_training = pd.concat([df_training, aez_onehot_df], axis=1)

## FEATURE SELECTION

In [48]:
# Features including AEZ (location) as primary feature
base_features = [
    # Location encoded (primary feature for Kenya's diverse climate)
    'aez_encoded',
    # Climate features from location
    'avg_temperature', 'avg_rainfall', 'avg_humidity', 'avg_solar_radiation',
    'temp_variability', 'rainfall_variability',
    # Crop requirement features
    'crop_temp_opt_min', 'crop_temp_opt_max',
    'crop_temp_abs_min', 'crop_temp_abs_max',
    'crop_rain_opt_min', 'crop_rain_opt_max',
    'crop_rain_abs_min', 'crop_rain_abs_max',
    'crop_ph_min', 'crop_ph_max',
    'growth_days_min', 'growth_days_max'
]


In [51]:
# Add one-hot encoded AEZ columns
aez_columns = [col for col in df_training.columns if col.startswith('aez_')]
feature_cols = base_features + aez_columns

print(f"Total features: {len(feature_cols)}")
print(f"Features: {feature_cols}")

X = df_training[feature_cols]
y = df_training['suitability']


Total features: 20
Features: ['aez_encoded', 'avg_temperature', 'avg_rainfall', 'avg_humidity', 'avg_solar_radiation', 'temp_variability', 'rainfall_variability', 'crop_temp_opt_min', 'crop_temp_opt_max', 'crop_temp_abs_min', 'crop_temp_abs_max', 'crop_rain_opt_min', 'crop_rain_opt_max', 'crop_rain_abs_min', 'crop_rain_abs_max', 'crop_ph_min', 'crop_ph_max', 'growth_days_min', 'growth_days_max', 'aez_encoded']


## Train Test Split

In [52]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set: {X_train.shape}")
print(f"Test set: {X_test.shape}")
print(f"\nTraining class distribution:\n{y_train.value_counts(normalize=True)}")


Training set: (4748, 20)
Test set: (1187, 20)

Training class distribution:
suitability
0    0.465249
1    0.417018
2    0.117734
Name: proportion, dtype: float64


In [53]:
# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## MODEL TRAINING

### Decision Tree

In [54]:
results = {}


print("\n--- Decision Tree ---")
dt = DecisionTreeClassifier(
    max_depth=15, 
    min_samples_split=10,
    min_samples_leaf=5,
    random_state=42
)
dt.fit(X_train_scaled, y_train)
y_pred_dt = dt.predict(X_test_scaled)

acc_dt = accuracy_score(y_test, y_pred_dt)
f1_dt = f1_score(y_test, y_pred_dt, average='weighted')
results['Decision Tree'] = {'accuracy': acc_dt, 'f1': f1_dt}
print(f"Accuracy: {acc_dt:.4f}")
print(f"F1-Score (weighted): {f1_dt:.4f}")



--- Decision Tree ---
Accuracy: 0.9832
F1-Score (weighted): 0.9832


### Random Forest

In [55]:
rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=15,
    min_samples_split=10,
    min_samples_leaf=5,
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train_scaled, y_train)
y_pred_rf = rf.predict(X_test_scaled)

acc_rf = accuracy_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf, average='weighted')
results['Random Forest'] = {'accuracy': acc_rf, 'f1': f1_rf}
print(f"Accuracy: {acc_rf:.4f}")
print(f"F1-Score (weighted): {f1_rf:.4f}")

Accuracy: 0.9764
F1-Score (weighted): 0.9764


### xgboost

In [56]:
xgb = XGBClassifier(
    n_estimators=100,
    max_depth=8,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)
xgb.fit(X_train_scaled, y_train)
y_pred_xgb = xgb.predict(X_test_scaled)

acc_xgb = accuracy_score(y_test, y_pred_xgb)
f1_xgb = f1_score(y_test, y_pred_xgb, average='weighted')
results['XGBoost'] = {'accuracy': acc_xgb, 'f1': f1_xgb}
print(f"Accuracy: {acc_xgb:.4f}")
print(f"F1-Score (weighted): {f1_xgb:.4f}")

Accuracy: 0.9992
F1-Score (weighted): 0.9992


## KFOLD CROSS VALIDATION

In [57]:
skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

### RANDOM FOREST CROSS VALIDATION

In [58]:
cv_acc_rf = cross_val_score(rf, X_train_scaled, y_train, cv=skfold, scoring='accuracy')
cv_f1_rf = cross_val_score(rf, X_train_scaled, y_train, cv=skfold, scoring='f1_weighted')
print(f"CV Accuracy: {cv_acc_rf.mean():.4f} (+/- {cv_acc_rf.std():.4f})")
print(f"CV F1-Score: {cv_f1_rf.mean():.4f} (+/- {cv_f1_rf.std():.4f})")



CV Accuracy: 0.9749 (+/- 0.0051)
CV F1-Score: 0.9746 (+/- 0.0053)


### xgboost cross validation

In [59]:
cv_acc_xgb = cross_val_score(xgb, X_train_scaled, y_train, cv=skfold, scoring='accuracy')
cv_f1_xgb = cross_val_score(xgb, X_train_scaled, y_train, cv=skfold, scoring='f1_weighted')
print(f"CV Accuracy: {cv_acc_xgb.mean():.4f} (+/- {cv_acc_xgb.std():.4f})")
print(f"CV F1-Score: {cv_f1_xgb.mean():.4f} (+/- {cv_f1_xgb.std():.4f})")

CV Accuracy: 0.9943 (+/- 0.0029)
CV F1-Score: 0.9943 (+/- 0.0029)


## Feature Importance Analysis

In [60]:
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': rf.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 15 Most Important Features:")
print(feature_importance.head(15).to_string(index=False))


Top 15 Most Important Features:
             feature  importance
   crop_rain_abs_min    0.162054
   crop_rain_abs_max    0.140996
   crop_rain_opt_min    0.089742
   crop_rain_opt_max    0.079543
rainfall_variability    0.051008
        avg_rainfall    0.049334
   crop_temp_opt_min    0.049002
   crop_temp_abs_min    0.044368
        avg_humidity    0.043454
     avg_temperature    0.039288
    temp_variability    0.039006
   crop_temp_opt_max    0.037935
         aez_encoded    0.037783
         aez_encoded    0.037298
   crop_temp_abs_max    0.030916


### check AEZ location importance

In [61]:
aez_features = feature_importance[
    feature_importance['feature'].str.contains('aez|AEZ', case=False)
]
print(f"\nAEZ (Location) Feature Importance:")
print(aez_features.to_string(index=False))
total_aez_importance = aez_features['importance'].sum()
print(f"\nTotal AEZ importance: {total_aez_importance:.4f} ({total_aez_importance*100:.1f}%)")



AEZ (Location) Feature Importance:
    feature  importance
aez_encoded    0.037783
aez_encoded    0.037298

Total AEZ importance: 0.0751 (7.5%)


## CLASSIFICATION REPORT

In [62]:
print(classification_report(y_test, y_pred_rf, 
                           target_names=['Not Suitable', 'Marginal', 'Highly Suitable']))


                 precision    recall  f1-score   support

   Not Suitable       1.00      0.98      0.99       552
       Marginal       0.96      0.99      0.97       495
Highly Suitable       0.95      0.91      0.93       140

       accuracy                           0.98      1187
      macro avg       0.97      0.96      0.96      1187
   weighted avg       0.98      0.98      0.98      1187



## selecting the best model

In [63]:
for model, metrics in results.items():
    print(f"  {model}: Accuracy={metrics['accuracy']:.4f}, F1={metrics['f1']:.4f}")

best_model = max(results.items(), key=lambda x: x[1]['f1'])
print(f"\nBest Model: {best_model[0]} (F1: {best_model[1]['f1']:.4f})")


  Decision Tree: Accuracy=0.9832, F1=0.9832
  Random Forest: Accuracy=0.9764, F1=0.9764
  XGBoost: Accuracy=0.9992, F1=0.9992

Best Model: XGBoost (F1: 0.9992)


## Saving the models with joblib compression

In [69]:
import os
os.makedirs('../models', exist_ok=True)

# Save models with compression for smaller file sizes (Render-friendly)
joblib.dump(rf, '../models/crop_recommendation_model.joblib', compress=5)
joblib.dump(xgb, '../models/crop_recommendation_xgb.joblib', compress=5)
joblib.dump(scaler, '../models/scaler_crop.joblib', compress=5)
joblib.dump(le_crop, '../models/crop_label_encoder.joblib', compress=5)
joblib.dump(le_aez, '../models/aez_label_encoder_crop.joblib', compress=5)
joblib.dump(ohe_aez, '../models/aez_onehot_encoder_crop.joblib', compress=5)
joblib.dump(feature_cols, '../models/crop_feature_columns.joblib', compress=5)


['../models/crop_feature_columns.joblib']